In [ ]:
# %pip install pandas
# %pip install pyspark

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, lit, col, sum as spark_sum
import os

In [23]:

class SparkPostgres:
    def __init__(self, spark, host, port, database, user, password):
        self.spark = spark
        self.url = f"jdbc:postgresql://{host}:{port}/{database}"
        self.properties = {
            "user": user,
            "password": password,
            "driver": "org.postgresql.Driver"
        }

    def load_csv(self, file_path, delimiter=';'):
        if os.path.exists(file_path):
            df = self.spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", delimiter).csv(file_path)
            return df
        else:
            print(f"Error: O arquivo {file_path} não existe.")
            return None

    def save_to_postgres(self, df, schema, table):
        if df is not None:
            df = df.withColumn('dt_pst', current_date())
            df.write.jdbc(
                url=self.url,
                table=f"{schema}.{table}",
                mode="overwrite",
                properties=self.properties
            )
        else:
            print(f"Error: DataFrame vazio para a tabela {schema}.{table}")

class ProcessandoDados:
    def __init__(self, spark_postgres):
        self.spark_postgres = spark_postgres

    def carregar_dados(self, file_path, delimiter=';'):
        """Carrega os dados de um arquivo CSV usando Spark."""
        return self.spark_postgres.load_csv(file_path, delimiter)

    def analisar_dados(self, df):
        """Exibe informações sobre os dados, como nulos, duplicados e tipos de colunas."""
        df.describe().show()
        df.select([df[col].isNull().sum().alias(col) for col in df.columns]).show()
        print("Quantidade de registros duplicados:", df.count() - df.dropDuplicates().count())
        df.printSchema()

    def process_columns(self, df, year):
        """Filtra colunas relevantes para um determinado ano."""
        cols = [col for col in df.columns if "NOME" in col or str(year) in col]
        df_filtered = df.select(cols).dropna(subset=[f"INDE_{year}"])
        for col in df_filtered.columns:
            df_filtered = df_filtered.withColumnRenamed(col, col.replace(f"_{year}", "").lower())
        df_filtered = df_filtered.withColumn("ano", lit(year))
        return df_filtered

    def process_year(self, df, year):
        """Processa os dados para um ano específico e retorna o dataframe correspondente."""
        return self.process_columns(df, year)

    def salvar_no_rds(self, df, schema, table):
        """Salva o dataframe em uma tabela no banco de dados RDS."""
        self.spark_postgres.save_to_postgres(df, schema, table)
        print(f"Dados salvos na tabela {schema}.{table} com sucesso.")

In [52]:
# usado para debug
df = None
processor = None
df_processed_2022 = None
df_processed_2021 = None
df_processed_2020 = None

if __name__ == "__main__":
    spark = SparkSession.builder.appName("CSV to PostgreSQL").getOrCreate()

    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")
    database = os.getenv("DB_DATABASE")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    schema = os.getenv("DB_SCHEMA")
    file_path = "./source/PEDE_PASSOS_DATASET_FIAP.csv"

    spark_postgres = SparkPostgres(spark, host, port, database, user, password)
    processor = ProcessandoDados(spark_postgres)
    
    df = processor.carregar_dados(file_path)
    for year in [2020, 2021, 2022]:
        df_processed = processor.process_columns(df, year)
        processor.salvar_no_rds(df_processed, schema, f"tb_dados_principais_{year}")
        print(f"\nDados processados para o ano {year} e salvos na tabela tb_dados_principais_{year}")
        
        # usado para debug
        match year:
            case 2020:
                df_processed_2020 = df_processed
            case 2021:
                df_processed_2021 = df_processed
            case 2021:
                df_processed_2022 = df_processed

Dados salvos na tabela techchallengefase5.tb_dados_principais_2020 com sucesso.

Dados processados para o ano 2020 e salvos na tabela tb_dados_principais_2020


Dados salvos na tabela techchallengefase5.tb_dados_principais_2021 com sucesso.

Dados processados para o ano 2021 e salvos na tabela tb_dados_principais_2021


Dados salvos na tabela techchallengefase5.tb_dados_principais_2022 com sucesso.

Dados processados para o ano 2022 e salvos na tabela tb_dados_principais_2022


In [57]:
df_processed_2022.show()

+------------------------+--------+-----------+-------+----------+------------+------------------+-------------+--------+--------------------+--------------------+--------------------+------------------+---+-----+-----------------+-----------------+-----------------+---+----+
|instituicao_ensino_aluno|    nome|idade_aluno|anos_pm|fase_turma|ponto_virada|              inde|inde_conceito|   pedra|        destaque_ieg|        destaque_ida|        destaque_ipv|               iaa|ieg|  ips|              ida|              ipp|              ipv|ian| ano|
+------------------------+--------+-----------+-------+----------+------------+------------------+-------------+--------+--------------------+--------------------+--------------------+------------------+---+-----+-----------------+-----------------+-----------------+---+----+
|          Escola Pública| ALUNO-1|         11|      2|        2H|         Não|          7.883752|            B|Ametista|Seu destaque em 2...|Ponto a melhorar ...|Seu de